In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

# Load the CSV file into a DataFrame
home_prices_df = pd.read_csv("Metro_US_All_Home_Prices.csv")

# Create a new column "Metro" by extracting the city name from "Region Name"
# We split the string in "Region Name" at the comma and take the first part (city)
home_prices_df["Metro"] = home_prices_df["RegionName"].str.split(',').str[0]

# Display the updated DataFrame to verify the new "Metro" column
home_prices_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,Metro
0,102001,0,United States,country,NaN,122710.838539,122926.726242,123194.458782,123768.692526,124429.566719,...,354242.717001,354677.065211,355473.042348,357021.050068,358884.957514,360324.712897,360888.668935,361037.327498,361281.717048,United States
1,394913,1,"New York, NY",msa,NY,218233.216724,219160.584763,220096.541902,221993.153236,223957.159952,...,639265.650987,642178.317205,646255.694516,651704.395720,657863.854229,662561.428749,665667.764258,668437.168324,672210.970325,New York
2,753899,2,"Los Angeles, CA",msa,CA,222104.914025,222931.426109,224032.332323,226222.765868,228618.413461,...,926554.539271,926708.887683,923740.986284,922618.466845,926105.996730,932034.718291,936012.934083,940417.087163,945635.978294,Los Angeles
3,394463,3,"Chicago, IL",msa,IL,153956.315498,154098.834197,154370.263570,155045.512395,155857.755512,...,311913.816691,312578.140750,313972.158826,316395.381263,319279.856054,321409.733463,322638.674660,323378.146509,324304.487366,Chicago
4,394514,4,"Dallas, TX",msa,TX,126111.301852,126167.472266,126232.045908,126399.736872,126620.832817,...,372112.028509,372133.731174,372616.126553,373774.335129,374743.310085,374887.158218,374222.635105,373283.837228,372632.458458,Dallas


In [5]:
commute_df = pd.read_csv("Commute_Time_By_Top_25_Metro_Area_All_Years__Three Buckets_Percentage.csv")
# Drop rows in the home_prices_df where "Metro" doesn't match with any city in "Metro" column of commute_df
filtered_home_prices_df = home_prices_df[home_prices_df["Metro"].isin(commute_df["Metro"])]
# Display the filtered DataFrame to verify the rows have been dropped correctly
filtered_home_prices_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,Metro
1,394913,1,"New York, NY",msa,NY,218233.216724,219160.584763,220096.541902,221993.153236,223957.159952,...,639265.650987,642178.317205,646255.694516,651704.395720,657863.854229,662561.428749,665667.764258,668437.168324,672210.970325,New York
2,753899,2,"Los Angeles, CA",msa,CA,222104.914025,222931.426109,224032.332323,226222.765868,228618.413461,...,926554.539271,926708.887683,923740.986284,922618.466845,926105.996730,932034.718291,936012.934083,940417.087163,945635.978294,Los Angeles
3,394463,3,"Chicago, IL",msa,IL,153956.315498,154098.834197,154370.263570,155045.512395,155857.755512,...,311913.816691,312578.140750,313972.158826,316395.381263,319279.856054,321409.733463,322638.674660,323378.146509,324304.487366,Chicago
4,394514,4,"Dallas, TX",msa,TX,126111.301852,126167.472266,126232.045908,126399.736872,126620.832817,...,372112.028509,372133.731174,372616.126553,373774.335129,374743.310085,374887.158218,374222.635105,373283.837228,372632.458458,Dallas
5,394692,5,"Houston, TX",msa,TX,121737.010654,121759.686522,121674.688937,121725.495460,121772.362850,...,304832.859556,304826.840219,305204.501474,306189.659687,307208.643453,307743.352455,307625.056921,307261.595181,307043.671704,Houston


In [6]:
commute_df = pd.read_csv("Commute_Time_By_Top_25_Metro_Area_All_Years__Three Buckets_Percentage.csv")
# Drop rows in the home_prices_df where "Metro City" doesn't match with any city in "Metro" column of commute_df
filtered_home_prices_df = home_prices_df[home_prices_df["Metro"].isin(commute_df["Metro"])]

# Display the filtered DataFrame to verify the rows have been dropped correctly
filtered_home_prices_df

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,Metro
1,394913,1,"New York, NY",msa,NY,218233.216724,219160.584763,220096.541902,221993.153236,223957.159952,...,6.392657e+05,6.421783e+05,6.462557e+05,6.517044e+05,6.578639e+05,6.625614e+05,6.656678e+05,6.684372e+05,6.722110e+05,New York
2,753899,2,"Los Angeles, CA",msa,CA,222104.914025,222931.426109,224032.332323,226222.765868,228618.413461,...,9.265545e+05,9.267089e+05,9.237410e+05,9.226185e+05,9.261060e+05,9.320347e+05,9.360129e+05,9.404171e+05,9.456360e+05,Los Angeles
3,394463,3,"Chicago, IL",msa,IL,153956.315498,154098.834197,154370.263570,155045.512395,155857.755512,...,3.119138e+05,3.125781e+05,3.139722e+05,3.163954e+05,3.192799e+05,3.214097e+05,3.226387e+05,3.233781e+05,3.243045e+05,Chicago
4,394514,4,"Dallas, TX",msa,TX,126111.301852,126167.472266,126232.045908,126399.736872,126620.832817,...,3.721120e+05,3.721337e+05,3.726161e+05,3.737743e+05,3.747433e+05,3.748872e+05,3.742226e+05,3.732838e+05,3.726325e+05,Dallas
5,394692,5,"Houston, TX",msa,TX,121737.010654,121759.686522,121674.688937,121725.495460,121772.362850,...,3.048329e+05,3.048268e+05,3.052045e+05,3.061897e+05,3.072086e+05,3.077434e+05,3.076251e+05,3.072616e+05,3.070437e+05,Houston
6,395209,6,"Washington, DC",msa,VA,180842.151942,180989.190625,181260.192650,181873.819130,182806.635695,...,5.501634e+05,5.505870e+05,5.520378e+05,5.550904e+05,5.592981e+05,5.627547e+05,5.644495e+05,5.650157e+05,5.655384e+05,Washington
7,394974,7,"Philadelphia, PA",msa,PA,122513.075912,122785.376616,122971.211007,123429.776479,123873.533424,...,3.530673e+05,3.542810e+05,3.558435e+05,3.581864e+05,3.609425e+05,3.630089e+05,3.639378e+05,3.643231e+05,3.650522e+05,Philadelphia
8,394856,8,"Miami, FL",msa,FL,112841.961497,113151.736319,113479.771936,114142.475317,114783.781240,...,4.780922e+05,4.804784e+05,4.824152e+05,4.845651e+05,4.865473e+05,4.882377e+05,4.888037e+05,4.889947e+05,4.890146e+05,Miami
9,394347,9,"Atlanta, GA",msa,GA,145114.913650,145449.378651,145843.440336,146681.566032,147617.899469,...,3.770676e+05,3.776150e+05,3.785426e+05,3.800731e+05,3.818430e+05,3.829845e+05,3.832073e+05,3.830043e+05,3.828614e+05,Atlanta
10,394404,10,"Boston, MA",msa,MA,216568.526977,217441.540329,218396.691528,220159.489052,222052.850426,...,6.696114e+05,6.715970e+05,6.742857e+05,6.789267e+05,6.849457e+05,6.900598e+05,6.927894e+05,6.943895e+05,6.960745e+05,Boston


In [7]:
# Step 1: Identify columns that contain date information
date_columns = filtered_home_prices_df.filter(regex=r'^\d{4}-\d{2}-\d{2}$').columns

# Step 2: Create a new DataFrame to store yearly averages
# Extract the year from the date columns
filtered_home_prices_df_yearly_avg = filtered_home_prices_df.copy()

# Loop through each year from the date columns
for year in range(2000, 2025):  # From 2000 to 2024
    # Select all columns that match the current year
    year_columns = [col for col in date_columns if col.startswith(str(year))]
    
    # Calculate the average for that year across the matching columns
    filtered_home_prices_df_yearly_avg[str(year)] = filtered_home_prices_df[year_columns].mean(axis=1)

# Drop the original date columns
filtered_home_prices_df_yearly_avg = filtered_home_prices_df_yearly_avg.drop(columns=date_columns)

# Display the updated DataFrame with yearly averages
filtered_home_prices_df_yearly_avg.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,Metro,2000,2001,2002,2003,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,394913,1,"New York, NY",msa,NY,New York,228236.333677,256591.123039,289406.051521,332275.171124,...,417767.278050,436687.392137,458570.405676,481006.500113,497671.075482,514606.039175,562982.333172,609970.348571,623831.364522,658359.949166
2,753899,2,"Los Angeles, CA",msa,CA,Los Angeles,232411.457365,257442.579993,291677.819819,346346.762012,...,507705.187043,527084.898482,573910.059070,633907.170969,649673.994925,683222.738515,778279.042016,882911.563806,881706.948498,931659.381922
3,394463,3,"Chicago, IL",msa,IL,Chicago,158276.964195,171890.368310,185654.507685,199856.902187,...,200373.019284,210856.095856,221825.942265,232372.941507,238138.587799,244847.236934,270610.015172,294727.817170,303240.128095,319244.572361
4,394514,4,"Dallas, TX",msa,TX,Dallas,127272.724323,129451.749818,132613.591398,136409.226463,...,182117.867220,203734.502389,224890.794380,243209.782563,251633.656933,262748.133755,304406.673419,369235.098967,371281.826100,373536.698994
5,394692,5,"Houston, TX",msa,TX,Houston,122403.123483,124028.932793,127026.323361,131259.404075,...,183534.022383,194909.774814,202110.803554,210371.454699,218684.752642,227258.825353,256617.624359,300703.437909,304271.847022,306637.915137


In [8]:
# Select the columns representing years between 2010 and 2024 (inclusive)
columns_to_keep = [str(year) for year in range(2010, 2025)]  # This creates a list of year columns from 2010 to 2024

# Keep only the required year columns and any non-year columns (like 'Metro', etc.)
# Assuming there are other necessary columns (e.g., 'Metro'), we include those as well
necessary_columns = ['Metro']  # Adjust this list to include any non-year columns you want to keep

# Filter the DataFrame to keep only the necessary and year columns
filtered_home_prices_df_yearly_avg = filtered_home_prices_df_yearly_avg[necessary_columns + columns_to_keep]

# Display the updated DataFrame
filtered_home_prices_df_yearly_avg.head()

,Metro,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,New York,396589.647630,384564.909211,375518.290765,384470.975667,404313.654609,417767.278050,436687.392137,458570.405676,481006.500113,497671.075482,514606.039175,562982.333172,609970.348571,623831.364522,658359.949166
2,Los Angeles,404119.916610,381628.225170,372390.499355,430129.125075,483187.074509,507705.187043,527084.898482,573910.059070,633907.170969,649673.994925,683222.738515,778279.042016,882911.563806,881706.948498,931659.381922
3,Chicago,197308.543137,179647.501490,167993.290058,175350.972220,192030.788324,200373.019284,210856.095856,221825.942265,232372.941507,238138.587799,244847.236934,270610.015172,294727.817170,303240.128095,319244.572361
4,Dallas,147029.550825,141360.812372,142964.048910,153243.281400,166007.231691,182117.867220,203734.502389,224890.794380,243209.782563,251633.656933,262748.133755,304406.673419,369235.098967,371281.826100,373536.698994
5,Houston,148284.112874,142123.906743,140945.029754,149056.617277,165669.306473,183534.022383,194909.774814,202110.803554,210371.454699,218684.752642,227258.825353,256617.624359,300703.437909,304271.847022,306637.915137


In [14]:
Home_Prices_Adjusted_df = pd.melt(filtered_home_prices_df_yearly_avg, id_vars=['Metro'], var_name='Year', value_name='Value')
Home_Prices_Adjusted_df = Home_Prices_Adjusted_df.sort_values(by=['Metro', 'Year']).reset_index(drop=True)
Home_Prices_Adjusted_df.head(30)


,Metro,Year,Value
0,Atlanta,2010,148689.100170
1,Atlanta,2011,131296.644196
2,Atlanta,2012,127172.806650
3,Atlanta,2013,141549.412391
4,Atlanta,2014,162089.948743
5,Atlanta,2015,174951.646311
6,Atlanta,2016,189197.874487
7,Atlanta,2017,204153.911849
8,Atlanta,2018,222524.201892
9,Atlanta,2019,237693.585081


In [20]:
import pandas as pd

# Assuming Home_Prices_Adjusted_df is already loaded
# If not, you'd need to load it first

# Sort the dataframe by Metro and Year
Home_Prices_Adjusted_df = Home_Prices_Adjusted_df.sort_values(['Metro', 'Year'])

# Group by Metro and calculate percent change
Change_In_Home_Prices_Adjusted_df = Home_Prices_Adjusted_df.groupby('Metro', group_keys=False).apply(
    lambda x: x.assign(
        PercentChange=x['Value'].pct_change().shift(-1) * 100
    )
).reset_index(drop=True)

# Format PercentChange to 5 decimal places
Change_In_Home_Prices_Adjusted_df['PercentChange'] = Change_In_Home_Prices_Adjusted_df['PercentChange'].apply(lambda x: f"{x:.5f}" if pd.notnull(x) else x)

# Remove the last year (2014) as it doesn't have a next year to compare to
Change_In_Home_Prices_Adjusted_df = Change_In_Home_Prices_Adjusted_df[Change_In_Home_Prices_Adjusted_df['Year'] != 2014]

# Display the first few rows of the new dataframe
print(Change_In_Home_Prices_Adjusted_df.head())

     Metro  Year          Value PercentChange
0  Atlanta  2010  148689.100170     -11.69720
1  Atlanta  2011  131296.644196      -3.14086
2  Atlanta  2012  127172.806650      11.30478
3  Atlanta  2013  141549.412391      14.51121
4  Atlanta  2014  162089.948743       7.93491


/var/folders/s5/q1t_tc_s20jg7hm1zgg3gvvc0000gn/T/ipykernel_19637/3747572703.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Change_In_Home_Prices_Adjusted_df = Home_Prices_Adjusted_df.groupby('Metro', group_keys=False).apply(


In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming Change_In_Home_Prices_Adjusted_df and commute_times_df are already loaded

# Merge datasets
merged_data = pd.merge(Change_In_Home_Prices_Adjusted_df, commute_times_df, on=['Metro', 'Year'], how='inner')

# Ensure 'Year' is numeric and filter for years 2010 through 2023
merged_data['Year'] = pd.to_numeric(merged_data['Year'], errors='coerce')
merged_data = merged_data[(merged_data['Year'] >= 2010) & (merged_data['Year'] <= 2023)]

# Prepare features
three_bucket_features = [
    'Short Commutes - Less than half-hour (%)',
    'Medium Commutes - Half-hour to one hour (%)',
    'Long Commutes - More than one hour (%)'
]
yes_no_feature = ['Yes Commute (more than 5 minutes) (%)']

# Function to run analysis
def run_analysis(X, y, feature_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'R-squared': r2,
        'MSE': mse,
        'Coefficients': dict(zip(feature_name, model.coef_)),
        'Intercept': model.intercept_
    }

# Analysis for three buckets
X_three_buckets = merged_data[three_bucket_features]
y = merged_data['PercentChange']
three_buckets_results = run_analysis(X_three_buckets, y, three_bucket_features)

# Analysis for Yes/No commute
X_yes_no = merged_data[yes_no_feature]
yes_no_results = run_analysis(X_yes_no, y, yes_no_feature)

# Print results
print("Results for Three Commute Buckets:")
print(f"R-squared: {three_buckets_results['R-squared']:.4f}")
print(f"Mean Squared Error: {three_buckets_results['MSE']:.4f}")
print("Coefficients:")
for feature, coef in three_buckets_results['Coefficients'].items():
    print(f"  {feature}: {coef:.4f}")
print(f"Intercept: {three_buckets_results['Intercept']:.4f}")

print("\nResults for Yes/No Commute:")
print(f"R-squared: {yes_no_results['R-squared']:.4f}")
print(f"Mean Squared Error: {yes_no_results['MSE']:.4f}")
print("Coefficient:")
for feature, coef in yes_no_results['Coefficients'].items():
    print(f"  {feature}: {coef:.4f}")
print(f"Intercept: {yes_no_results['Intercept']:.4f}")

Results for Three Commute Buckets:
R-squared: -0.0672
Mean Squared Error: 1469.0717
Coefficients:
  Short Commutes - Less than half-hour (%): -19897.0778
  Medium Commutes - Half-hour to one hour (%): -19894.0470
  Long Commutes - More than one hour (%): -19898.3521
Intercept: 1989627.6724

Results for Yes/No Commute:
R-squared: -0.0041
Mean Squared Error: 1382.2544
Coefficient:
  Yes Commute (more than 5 minutes) (%): 20.4775
Intercept: -1998.3434


In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans

# Assuming merged_data is already prepared as before

# Prepare features and target
features = ['Short Commutes - Less than half-hour (%)', 
            'Medium Commutes - Half-hour to one hour (%)', 
            'Long Commutes - More than one hour (%)',
            'Yes Commute (more than 5 minutes) (%)']
X = merged_data[features]
y = merged_data['PercentChange']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to evaluate model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

# 1. Polynomial Regression
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
X_poly_train, X_poly_test = train_test_split(X_poly, test_size=0.2, random_state=42)
poly_model = LinearRegression()
poly_mse, poly_r2 = evaluate_model(poly_model, X_poly_train, X_poly_test, y_train, y_test)

# 2. Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_mse, rf_r2 = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

# 3. Lasso Regression
lasso_model = Lasso(alpha=1.0)
lasso_mse, lasso_r2 = evaluate_model(lasso_model, X_train, X_test, y_train, y_test)

# 4. Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_mse, ridge_r2 = evaluate_model(ridge_model, X_train, X_test, y_train, y_test)

# 5. Clustering + Regression
kmeans = KMeans(n_clusters=3, random_state=42)
merged_data['Cluster'] = kmeans.fit_predict(X)
cluster_results = []
for cluster in range(3):
    cluster_data = merged_data[merged_data['Cluster'] == cluster]
    X_cluster = cluster_data[features]
    y_cluster = cluster_data['PercentChange']
    X_train, X_test, y_train, y_test = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=42)
    cluster_model = LinearRegression()
    cluster_mse, cluster_r2 = evaluate_model(cluster_model, X_train, X_test, y_train, y_test)
    cluster_results.append((cluster, cluster_mse, cluster_r2))

# Print results
print("Polynomial Regression - MSE: {:.4f}, R-squared: {:.4f}".format(poly_mse, poly_r2))
print("Random Forest - MSE: {:.4f}, R-squared: {:.4f}".format(rf_mse, rf_r2))
print("Lasso Regression - MSE: {:.4f}, R-squared: {:.4f}".format(lasso_mse, lasso_r2))
print("Ridge Regression - MSE: {:.4f}, R-squared: {:.4f}".format(ridge_mse, ridge_r2))
print("Clustering + Regression Results:")
for cluster, mse, r2 in cluster_results:
    print("Cluster {}: MSE: {:.4f}, R-squared: {:.4f}".format(cluster, mse, r2))

# Feature importance for Random Forest
feature_importance = pd.DataFrame({'feature': features, 'importance': rf_model.feature_importances_})
print("\nRandom Forest Feature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Polynomial Regression - MSE: 1422.4838, R-squared: -0.0333
Random Forest - MSE: 3472.7205, R-squared: -1.5227
Lasso Regression - MSE: 1435.4495, R-squared: -0.0428
Ridge Regression - MSE: 1432.2625, R-squared: -0.0404
Clustering + Regression Results:
Cluster 0: MSE: 8518.6208, R-squared: -0.0123
Cluster 1: MSE: 58.1880, R-squared: -0.4406
Cluster 2: MSE: 7.0388, R-squared: -0.5436

Random Forest Feature Importance:
                                       feature  importance
3        Yes Commute (more than 5 minutes) (%)    0.340030
2       Long Commutes - More than one hour (%)    0.278600
1  Medium Commutes - Half-hour to one hour (%)    0.232509
0     Short Commutes - Less than half-hour (%)    0.148861
